# Test model 2, input your file you want to load

## Imports

In [ ]:
import os
from glob import glob
from math import floor
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, Input, UpSampling2D
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Reshape, MaxPooling2D,  MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model
from tensorflow.keras.backend import int_shape
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg19 import VGG19

### Set up working dirsand paths, PLEASE chdir to your path!!!!

In [ ]:
#SET UP HERE CHANGE THE VALUE TO YOUR FILE DIRECTORY
os.chdir('C:/Users/YOUR_USER/Documents/Colourising_Photos/')
height, width = (256,256)
#Setting up path for data
inputPath = os.getcwd()+"/example_inputs/p_256/*.jpg"
outputPath = (os.getcwd()+"/Test_output/Model1")

## Create the model to load our weights into

In [ ]:
vgg19 = VGG19(include_top=False, weights='imagenet' ,input_tensor=None,  pooling=None, input_shape=(width,height,3), classes=1000)

In [ ]:

#Create input, take 256x256 photographs, this can be changed at a later date.
inputs = Input(name='Model2_Input', shape=(256,256,1)  )
x = Conv2D(filters=3,kernel_size=3, activation='relu',padding='same')(inputs)

#Input part of the VGG19 model, not all layers.
for lyr in vgg19.layers[:-13]:
    x = lyr(x)
    x.trainable=False

#Perform two transpose conv's for 128x128,128, and 256x256,3 shapes
x = Conv2DTranspose(kernel_size=3,activation='relu',strides = 2 , padding='same',filters=128)(x)
x = Conv2DTranspose(kernel_size=3,activation='relu',strides = 2 , padding='same',filters=3)(x)

#finish the model
model = Model(inputs, x, name='model2')

#Load all of our weights
model.load_weights(""+os.getcwd()+"/Model2/model2_920.h5")
#Compile using MSE for loss, and adam optimiser
model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])


### Input data to colourise

In [ ]:
colourise = []

pics = glob(inputPath)
index = 0;

for p in pics:
    #Take photos by value
    picName = pics[index].split('\\')[1].split('.')[0]
    img = Image.open(pics[index])
    index += 1
     
    imgBW = img.convert('L')  

    BW = np.array(imgBW).astype('float32')
    BW /= 255
    colourise.append(BW)
    img.close()

colourise = np.asarray(colourise).flatten()
colourise = colourise.reshape(-1,width,height,1)


### Outputs Data to /test_output/ file

In [ ]:
# Predict
output = model.predict(colourise[:])
#Now return the values from 0->1 to 0->255
output *= 255
#Convert from f_32 -> u_int8
output = output.astype('uint8')
#save train images
photo_num = 1
for img in output:
    output = Image.fromarray(img)
    output.save(''+os.getcwd()+'/Test_output/Model2/ouput{}.png'.format(photo_num))
    photo_num += 1